In [ ]:
from ltr.clickmodels.session import build

doc_costs = {
    0: 0.4,
    1: 0.5,
    2: 0.5,
    3: 0.4,
    4: 0.7,
    5: 0.5,
    6: 0.5,
    7: 0.4,
    8: 0.7,
    9: 0.5,
}

# Sessions for search queries "rambo" and "star trek"
# First bool click, second click is a conversion (empty implies no conversion)
#
# Here 1 is the 'good guy' that always gets a conversion
#      3 gets clicks, but never gets a conversion
sessions = build([
      ('rambo',     ((1, True, True), (2, False), (3, False), (0, False))),
      ('rambo',     ((1, False), (2, True, True), (3, True), (0, False))),
      ('rambo',     ((4, False), (2, False), (1, True, True), (0, True))),
      ('rambo',     ((7, False), (4, False), (1, False), (3, False))),
      ('rambo',     ((1, False), (8, False), (2, False), (3, False))),
      ('rambo',     ((3, True), (2, False), (1, False), (0, False))),
      ('rambo',     ((3, True), (2, False), (1, False), (0, False))),
      ('rambo',     ((9, False), (3, True), (1, True,True), (0, True))),
      ('rambo',     ((6, False), (4, True, True), (1, False), (3, False))),
      ('rambo',     ((0, True,True), (2, True), (1, True,True), (4, True))),


    ])

In [ ]:
from ltr.clickmodels.pbm import position_based_model
model = position_based_model(sessions, rounds=50)
model.attracts

In [ ]:
model.ranks

In [ ]:
from ltr.clickmodels.conversion import conv_aug_attracts
satisfacts = conv_aug_attracts(model.attracts, sessions, doc_costs)
satisfacts

In [ ]:
def by_query(judgment_dict):
    from collections import defaultdict
    by_query_ranked = defaultdict(list)
    
    for (query, doc), score in judgment_dict.items():
        by_query_ranked[query].append((doc,score))
        
    for query, items in by_query_ranked.items():
        items.sort(key=lambda score_doc: score_doc[1], reverse=True)
        
    return by_query_ranked

def just_doc_ids(docs):
    return [doc[0] for doc in docs]


s_satisfacts = by_query(satisfacts)
s_attracts = by_query(model.attracts)

s_satisfacts

In [ ]:
s_attracts

In [ ]:
from ltr.helpers.tau import tau, avg_tau

tau(just_doc_ids(s_satisfacts['rambo']),just_doc_ids(s_attracts['rambo']))